In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision import models
import torch.nn.functional as F

train_set = datasets.MNIST(root = 'data',
                           train = True,
                           download = True,
                           transform = transforms.ToTensor())

test_set = datasets.MNIST(root = 'data',
                          train = False,
                          download = True,
                          transform = transforms.ToTensor())

class MLP(nn.Module) :
    def __init__(self):
        super(MLP, self).__init__()
        self.func1 = nn.Linear(28 * 28, 200)
        self.func2 = nn.Linear(200, 200)
        self.func3 = nn.Linear(200, 200)
        self.func4 = nn.Linear(200, 200)
        self.func5 = nn.Linear(200, 200)
     
        
        self.func15 = nn.Linear(200, 10)
        
    def forward(self, x):
        x = x.view(len(x), -1)
        x = self.func1(x)
        
        x = self.func2(x)
        
        x = self.func3(x)
        
        x = self.func4(x)
        
        x = self.func5(x)
        
        
        
        x = self.func15(x)
        
        return F.softmax(x)

# def weights_init(m) :
#     if isinstance(m, (nn.Linear, nn.Conv2d)) :
#         nn.init.uniform_(m.weight)
#         nn.init.constant_(m.bias, 0.0)

Batch_size = 64
Max_epoch = 30
learning_rate = 0.01
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model = MLP().to(device)
# model.apply(weights_init)
print(model)

train_loader = DataLoader(train_set, batch_size = Batch_size, shuffle = True, num_workers = 0)
test_loader = DataLoader(test_set, batch_size = Batch_size, shuffle = False, num_workers = 0)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), momentum = 0.9, lr = learning_rate)

train_losses = []
train_acces = []

for epoch in range(Max_epoch) :
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    for input, label in train_loader :
#         print(input.size())
        input, label = input.to(device), label.to(device)

        optimizer.zero_grad()
        outputs = model(input)

        loss = criterion(outputs, label)
        train_loss += loss.item()
        loss.backward()

        _, pred = torch.max(outputs, dim = 1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / input.size(0)
        train_acc += acc

        optimizer.step()
    with torch.no_grad():
        model.eval()
        total_loss = 0.0
        test_acc = 0.0
        for input, label in test_loader:
            input, label = input.to(device), label.to(device)

            outputs = model(input)
            loss = criterion(outputs, label)
            total_loss += loss.item()

            _, pred = torch.max(outputs, dim=1)
            num_correct = (pred == label).sum().item()
            acc = num_correct / input.size(0)
            test_acc += acc

    train_losses.append(train_loss / len(train_loader))
    train_acces.append(train_acc / len(train_loader))

    print(f'epoch : {epoch + 1}, train_loss : {train_loss / len(train_loader)}, '
          f'train_acc : {train_acc / len(train_loader)}, '
          f'test_loss : {total_loss / len(test_loader)}, '
          f'test_acc : {test_acc / len(test_loader)}')

MLP(
  (func1): Linear(in_features=784, out_features=200, bias=True)
  (func2): Linear(in_features=200, out_features=200, bias=True)
  (func3): Linear(in_features=200, out_features=200, bias=True)
  (func4): Linear(in_features=200, out_features=200, bias=True)
  (func5): Linear(in_features=200, out_features=200, bias=True)
  (func15): Linear(in_features=200, out_features=10, bias=True)
)


<ipython-input-7-0b1483c8a7d3>:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


epoch : 1, train_loss : 2.1676945968477455, train_acc : 0.3392857142857143, test_loss : 1.765587910725053, test_acc : 0.7057125796178344
epoch : 2, train_loss : 1.6933007298756255, train_acc : 0.7726379264392325, test_loss : 1.658755903790711, test_acc : 0.8039410828025477
epoch : 3, train_loss : 1.647231240897799, train_acc : 0.8151652452025586, test_loss : 1.6297420415149373, test_acc : 0.8328025477707006
epoch : 4, train_loss : 1.634789612501669, train_acc : 0.8265258528784648, test_loss : 1.6247170480193607, test_acc : 0.8366839171974523
epoch : 5, train_loss : 1.6323626504015567, train_acc : 0.8283582089552238, test_loss : 1.6246598769145406, test_acc : 0.8361863057324841
epoch : 6, train_loss : 1.6311687198020757, train_acc : 0.8294243070362474, test_loss : 1.6284156863097172, test_acc : 0.8317078025477707
epoch : 7, train_loss : 1.6301706834896794, train_acc : 0.8306736407249466, test_loss : 1.6240186653319437, test_acc : 0.8369824840764332
epoch : 8, train_loss : 1.631274414469

KeyboardInterrupt: 